In [1]:
from config.config import cfg
from model.model import VisualGroundingModel
from data.dataset import VisualGroundingDataset
from data.preprocessor import PreProcessor

In [2]:
import os
from config.config import cfg
import torch
from torch.utils.data import DataLoader

In [3]:
vgmodel = VisualGroundingModel(cfg, cfg['training']['n_gpu']>1)

In [4]:
preprocessor = PreProcessor(cfg)
train_dataset = VisualGroundingDataset('train', preprocessor)
valid_dataset = VisualGroundingDataset('valid', preprocessor)

In [5]:
train_loader = DataLoader(train_dataset, collate_fn=train_dataset.preprocessor.collate,
                                       batch_size=cfg['training']['batch_size'], shuffle=False, num_workers=cfg['training']['num_workers'])
valid_loader = DataLoader(valid_dataset, collate_fn=valid_dataset.preprocessor.collate,
                                       batch_size=cfg['training']['batch_size'], shuffle=False, num_workers=cfg['training']['num_workers'])


In [6]:
vgmodel.to('cuda')
vgmodel.eval()

VisualGroundingModel(
  (text_features): TextualFeatures(
    (lstm_1): LSTM(1024, 512, batch_first=True, bidirectional=True)
    (lstm_2): LSTM(1024, 512, batch_first=True, bidirectional=True)
    (leaky_relu): LeakyReLU(negative_slope=0.25)
    (sentence_fc): Sequential(
      (0): Linear(in_features=1024, out_features=1024, bias=True)
      (1): LeakyReLU(negative_slope=0.25)
      (2): Linear(in_features=1024, out_features=1024, bias=True)
      (3): LeakyReLU(negative_slope=0.25)
    )
    (word_fc): Sequential(
      (0): Linear(in_features=1024, out_features=1024, bias=True)
      (1): LeakyReLU(negative_slope=0.25)
      (2): Linear(in_features=1024, out_features=1024, bias=True)
      (3): LeakyReLU(negative_slope=0.25)
    )
    (word_linear_comb): Linear(in_features=3, out_features=1, bias=False)
    (sentence_linear_comb): Linear(in_features=2, out_features=1, bias=False)
  )
  (visual_features): VisualFeatures(
    (0_conv_0): Conv2d(256, 1024, kernel_size=(1, 1), stride=(

In [7]:
for i in train_loader:
#     print(i)
    print(i[0].shape)
    print(i[1].shape)
    print(i[-1])
    break

torch.Size([8, 4, 256, 18, 18])
torch.Size([8, 19, 1024])
tensor([[19],
        [17],
        [16],
        [16],
        [16],
        [14],
        [14],
        [11]])


In [13]:
i[0][0].shape # First item in batch

torch.Size([4, 256, 18, 18])

In [8]:
output = vgmodel(i[0], i[1], i[-1])

In [9]:
len(output)

4

In [10]:
output[1][0].shape

torch.Size([8, 8, 18, 18, 1, 1])

In [17]:
from PIL import Image
import torchvision.transforms as T
transform = T.ToPILImage()

In [18]:
img = transform(output[0][0])

ValueError: pic should be 2/3 dimensional. Got 6 dimensions.

In [14]:
import pytorch_lightning as pl

In [18]:
trainer = pl.Trainer(
            devices=cfg['training']['n_gpu'],
            accelerator="gpu",
            # strategy=self.cfg['training']['accelerator'] if self.distributed else None,
            strategy=None,
            accumulate_grad_batches=cfg['training']['accumulate_grad_batches'],
            max_epochs=cfg['training']['epochs'],
            default_root_dir=cfg['training']['local_trained_model_path'],
            log_every_n_steps=cfg['training']['log_every_n_steps']
        )

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(vgmodel, train_loader, valid_loader)

Missing logger folder: /models/trained_models_resnet/lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]

  | Name            | Type                | Params
--------------------------------------------------------
0 | text_features   | TextualFeatures     | 16.8 M
1 | visual_features | VisualFeatures      | 9.4 M 
2 | attention       | MultimodalAttention | 0     
3 | loss            | MultimodalLoss      | 0     
--------------------------------------------------------
26.2 M    Trainable params
0         Non-trainable params
26.2 M    Total params
104.989   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/opt/conda/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 40 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Sanity Checking DataLoader 0:   0%|                                                                                                                                    | 0/2 [00:03<?, ?it/s]

/opt/conda/lib/python3.8/site-packages/pytorch_lightning/utilities/data.py:72: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 8. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(


/opt/conda/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 40 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 0:   0%|                                                                                                                    | 5/156261 [00:17<148:08:56,  3.41s/it, loss=66.3, v_num=0]